In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"

In [2]:
import addict
import copy
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm
from typing import List, Dict, Tuple

import celeri

plt.rcParams["text.usetex"] = False # Plotting the global model is much much faster with tex fonts turned off

# Read in data files, create storage dictionaries, and do basic processing

In [4]:
# Japan example
command_file_name = "../data/command/japan_command.json"

# Western North America example
# command_file_name = "../data/command/western_north_america_command.json"

In [13]:
from scipy.spatial.distance import cdist

command = celeri.get_command(command_file_name)
celeri.create_output_folder(command)
logger = celeri.get_logger(command)
segment, block, meshes, station, mogi, sar = celeri.read_data(command)

segment = celeri.snap_segments(segment, meshes)

# # For each mesh, find associated segments
# cut_segment_idx = []
# all_edge_segment = celeri.make_default_segment(0)
# for i in range(len(meshes)):
#     these_segments = np.where((segment.patch_flag != 0) & (segment.patch_file_name == i+1))[0]
#     cut_segment_idx = np.append(cut_segment_idx, these_segments)
#     # Get top coordinates of the mesh
#     top_el_indices = np.where(meshes[i].top_elements)
#     edges = (np.sort(meshes[i].ordered_edge_nodes[:-1], axis=1))
#     top_verts = (np.sort(meshes[i].verts[top_el_indices], axis=1))
#     # Concatenate edges with vertex pairs
#     edges1 = np.vstack((edges, top_verts[:, 0:2]))
#     # Find unique edges
#     unique_edges1, unique_indices1, unique_counts1 = np.unique(edges1, axis=0, return_index=True, return_counts=True)
#     # But keep those edges that appear twice
#     top_edge_indices1 = unique_indices1[np.where(unique_counts1==2)]
#     # Same process with 2nd and 3rd columns of the mesh vertex array
#     edges2 = np.vstack((edges, top_verts[:, 1:3]))
#     unique_edges2, unique_indices2, unique_counts2 = np.unique(edges2, axis=0, return_index=True, return_counts=True)
#     top_edge_indices2 = unique_indices2[np.where(unique_counts2==2)]
#     # Final selection
#     top_edge_indices = np.sort(np.hstack((top_edge_indices1, top_edge_indices2)))
#     # Get new segment coordinates from these indices
#     edge_segs = celeri.make_default_segment(len(top_edge_indices))
#     edge_segs.lon1 = meshes[i].meshio_object.points[meshes[i].ordered_edge_nodes[top_edge_indices, 0], 0]
#     edge_segs.lat1 = meshes[i].meshio_object.points[meshes[i].ordered_edge_nodes[top_edge_indices, 0], 1]
#     edge_segs.lon2 = meshes[i].meshio_object.points[meshes[i].ordered_edge_nodes[top_edge_indices, 1], 0]
#     edge_segs.lat2 = meshes[i].meshio_object.points[meshes[i].ordered_edge_nodes[top_edge_indices, 1], 1]
#     edge_segs.locking_depth =- 15
#     edge_segs.patch_flag =+ 1
#     edge_segs.patch_file_name =+ i+1
#     all_edge_segment = all_edge_segment.append(edge_segs)

# # Get indices of segments to keep
# keep_segment_idx = np.setdiff1d(range(len(segment.lon1)), cut_segment_idx)
# # Isolate kept segments and reindex
# keep_segment = segment.loc[keep_segment_idx]
# new_index = range(len(keep_segment_idx))
# keep_segment.index = new_index
# # Find hanging endpoints; these mark terminations of mesh-replaced segments
# lons = np.hstack((keep_segment.lon1, keep_segment.lon2))
# lats = np.hstack((keep_segment.lat1, keep_segment.lat2))
# coords = np.array([lons, lats])
# unique_coords, indices, counts = np.unique(coords, axis=1, return_index=True, return_counts=True)
# hanging_idx = indices[np.where(counts == 1)]
# # Calculate distance to all mesh edge coordinates 
# # Can't just use the terminations because we might have triple junctions in the middle of a mesh
# elons = np.hstack((all_edge_segment.lon1, all_edge_segment.lon2))
# elats = np.hstack((all_edge_segment.lat1, all_edge_segment.lat2))
# ecoords = np.array([elons, elats])
# hang_to_mesh_dist = cdist(coords[:,hanging_idx].T, ecoords.T)
# # Find closest edge coordinate
# closest_edge_idx = np.argmin(hang_to_mesh_dist, axis=1)
# # Replace segment coordinates with closest mesh coordinate
# # Using a loop because we need to evaluate whether to replace endpoint 1 or 2
# for i in range(len(closest_edge_idx)):
#     if hanging_idx[i] < len(keep_segment.lon1):
#         keep_segment.loc[hanging_idx[i], 'lon1'] = ecoords[0, closest_edge_idx[i]]
#         keep_segment.loc[hanging_idx[i], 'lat1'] = ecoords[1, closest_edge_idx[i]]
#     else:
#         keep_segment.loc[hanging_idx[i]-len(keep_segment.lon1), 'lon2'] = ecoords[0, closest_edge_idx[i]]
#         keep_segment.loc[hanging_idx[i]-len(keep_segment.lon1), 'lat2'] = ecoords[1, closest_edge_idx[i]]    
# # Merge with mesh edge segments
# new_segments = keep_segment.append(all_edge_segment)
# new_segments.to_csv('~/Desktop/file1.csv')

    
    

2022-05-18 10:59:26.257 | INFO     | celeri.celeri:get_logger:107 - Read: ../data/command/japan_command.json
2022-05-18 10:59:26.257 | INFO     | celeri.celeri:get_logger:108 - RUN_NAME: 2022-05-18-10-59-26
2022-05-18 10:59:26.257 | INFO     | celeri.celeri:get_logger:109 - Write log file: ../runs/2022-05-18-10-59-26/2022-05-18-10-59-26.log
2022-05-18 10:59:26.258 | INFO     | celeri.celeri:read_data:114 - Reading data files
2022-05-18 10:59:26.261 | SUCCESS  | celeri.celeri:read_data:118 - Read: ../data/segment/japan_segment.csv
2022-05-18 10:59:26.263 | SUCCESS  | celeri.celeri:read_data:123 - Read: ../data/block/japan_block.csv
2022-05-18 10:59:26.264 | SUCCESS  | celeri.celeri:read_data:130 - Read: ../data/mesh/japan_mesh_parameters.json


Warning: The file contains tag data that couldn't be processed.

2022-05-18 10:59:26.286 | SUCCESS  | celeri.celeri:read_data:236 - Read: ../data/mesh/nankai.msh


Warning: The file contains tag data that couldn't be processed.

2022-05-18 10:59:26.325 | SUCCESS  | celeri.celeri:read_data:236 - Read: ../data/mesh/japan.msh


Warning: The file contains tag data that couldn't be processed.

2022-05-18 10:59:26.358 | SUCCESS  | celeri.celeri:read_data:236 - Read: ../data/mesh/sagami.msh
2022-05-18 10:59:26.362 | SUCCESS  | celeri.celeri:read_data:272 - Read: ../data/station/japan_station.csv
2022-05-18 10:59:26.363 | INFO     | celeri.celeri:read_data:287 - No mogi_file_name
2022-05-18 10:59:26.364 | INFO     | celeri.celeri:read_data:309 - No sar_file_name


In [ ]:
command = celeri.get_command(command_file_name)
celeri.create_output_folder(command)
logger = celeri.get_logger(command)
segment, block, meshes, station, mogi, sar = celeri.read_data(command)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
assembly = addict.Dict()
operators = addict.Dict()
operators.meshes = [addict.Dict()] * len(meshes)
assembly = celeri.merge_geodetic_data(assembly, station, sar) # Not sure this works correctly

# Get elastic operators and TDE smoothing operators

In [ ]:
# Get all elastic operators for segments and TDEs
celeri.get_elastic_operators(operators, meshes, segment, station, command)

# Get TDE smoothing operators
celeri.get_all_mesh_smoothing_matrices(meshes, operators)

# Calculate non-elastic operators

In [ ]:
operators.rotation_to_velocities = celeri.get_rotation_to_velocities_partials(station)
operators.global_float_block_rotation = celeri.get_global_float_block_rotation_partials(station)
assembly, operators.block_motion_constraints = celeri.get_block_motion_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.get_slip_rate_constraints(assembly, segment, block, command)
operators.rotation_to_slip_rate = celeri.get_rotation_to_slip_rate_partials(segment, block)
operators.block_strain_rate_to_velocities, strain_rate_block_index = celeri.get_block_strain_rate_to_velocities_partials(block, station, segment)
operators.mogi_to_velocities = celeri.get_mogi_to_velocities_partials(mogi, station, command)
celeri.get_tde_slip_rate_constraints(meshes, operators)

# Plot input data

In [ ]:
celeri.plot_input_summary(command, segment, station, block, meshes, mogi, sar, lon_range=(130.0, 150.0), lat_range=(30.0, 45.0), quiver_scale=1e2)

# Snapping segments to conform to mesh geometries

In [ ]:
default_segment = celeri.make_default_segment(5)
print(default_segment)

# Sketching out the assembly of the block model system

$$
\begin{bmatrix}
    \mathrm{geodetic \; velocities} \\
    \mathrm{plate \; rotation \; constraints} \\
    \mathrm{slip \; rate \; constraints} \\
    \mathrm{TDE \; smoothing \; pseudodata = 0} \\
    \mathrm{TDE \; rate \; constraints} \\
    \mathrm{InSAR \; LOS \; changes} 
\end{bmatrix}
=
\begin{bmatrix}
    \mathrm{(rotations-elastic \; segments) \; to \; velocities} & \mathrm{TDEs \; to \; velocities} & \mathrm{block \; strain \; rate \; to \; velocities} & \mathrm{Mogi \; to \; velocities}\\
    \mathrm{identities}                                          & 0                                   & 0 \\
    \mathrm{plate \; rotations \; to \; slip \; rates}           & 0                                   & 0 \\
    0                                                            & \mathrm{TDE \; smoothing \; matrix} & 0 \\
    0                                                            & \mathrm{identities}                 & 0 \\
    \mathrm{(rotations-elastic \; segments) \; to \; LOS}        & \mathrm{TDEs \; to \; LOS}          & \mathrm{block \; strain \; rate \; to \; velocities}
\end{bmatrix}
\begin{bmatrix}
    \mathrm{plate \; rotation \; rates} \\
    \mathrm{TDE \; slip \; rates} \\
    \mathrm{block \; strain \; rates} \\
    \mathrm{Mogi \; rates}
\end{bmatrix}
$$

# Estimate block model parameters (dense)

In [ ]:
index, estimation = celeri.assemble_and_solve_dense(command, assembly, operators, station, block, meshes)
celeri.post_process_estimation(estimation, operators, station, index)
celeri.write_output(command, estimation, station, segment, block, meshes)

# Plot model summary

In [ ]:
celeri.plot_estimation_summary(command, segment, station, meshes, estimation, lon_range=(130.0, 150.0), lat_range=(30.0, 45.0), quiver_scale=1e2)